<a href="https://colab.research.google.com/github/SJY-coder/TSP-demo/blob/master/TSP_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fetch Kaggle Datasets into Colab**

In [138]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [139]:
import os
os.environ['KAGGLE_CONFIG_DIR']="/content/gdrive/My Drive/Kaggle"

# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

#changing the working directory
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
#only for the first runtime
# !kaggle datasets download -d aaron7sun/stocknews

In [ ]:
!ls

Combined_News_DJIA.csv	RedditNews.csv	upload_DJIA_table.csv
kaggle.json		stocknews.zip


In [ ]:
#only for the first runtime
# !unzip \*.zip && rm *.zip

In [ ]:
!ls

Combined_News_DJIA.csv	RedditNews.csv	upload_DJIA_table.csv
kaggle.json		stocknews.zip


#**Preprocessing Datasets**

### merge 'Combined_News_DJIA.csv' with 'upload_DJIA_table.csv' to predict precise open&close price for the next day.


In [140]:
import pandas as pd
df1 = pd.read_csv('Combined_News_DJIA.csv')
df2 = pd.read_csv('upload_DJIA_table.csv')
#merge on 'Date' column for left&right
full_df = df1.merge(df2, left_on = 'Date', right_on = 'Date')

In [141]:
print(df1.shape,"+",df2.shape,"=>",full_df.shape)
full_df.head()

(1989, 27) + (1989, 7) => (1989, 33)


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',b'Rice Gives Green Light for Israel to Attack ...,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,"b""Georgia's move, a mistake of monumental prop...",b'Russia presses deeper into Georgia; U.S. say...,b'Abhinav Bindra wins first ever Individual Ol...,b' U.S. ship heads for Arctic to define territ...,b'Drivers in a Jerusalem taxi station threaten...,b'The French Team is Stunned by Phelps and the...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...","b""The US military was surprised by the timing ...",b'U.S. Beats War Drum as Iran Dumps the Dollar',"b'Gorbachev: ""Georgian military attacked the S...",b'CNN use footage of Tskhinvali ruins to cover...,b'Beginning a war as the Olympics were opening...,b'55 pyramids as large as the Luxor stacked in...,b'The 11 Top Party Cities in the World',b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS 

In [142]:
import numpy as np
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None
#giving df a new column of nexts day's open price & close price for target + text preprocessing
str_df = full_df.iloc[:,2:27].astype(str)
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
  """custom function to remove punctuation"""
  return text.translate(str.maketrans('', '',PUNCT_TO_REMOVE))
def remove_b(str_df): 
  for i in range(str_df.shape[1]):
    str_df.iloc[:,i] = str_df.iloc[:,i].apply(lambda text: remove_punctuation(text))
    for j in range(len(str_df.iloc[:,1])):
      #deleting 'b' in front of sentence
      str_df.iloc[:,i][j] = str_df.iloc[:,i][j][1:]
  return str_df
remove_b(str_df)
full_df.iloc[:,2:27] = str_df
full_df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close
0,2008-08-08,0,Georgia downs two Russian warplanes as countri...,BREAKING Musharraf to be impeached,Russia Today Columns of troops roll into South...,Russian tanks are moving towards the capital o...,Afghan children raped with impunity UN officia...,150 Russian tanks have entered South Ossetia w...,Breaking Georgia invades South Ossetia Russia ...,The enemy combatent trials are nothing but a s...,Georgian troops retreat from S Osettain capita...,Did the US Prep Georgia for War with Russia,Rice Gives Green Light for Israel to Attack Ir...,AnnouncingClass Action Lawsuit on Behalf of Am...,SoRussia and Georgia are at war and the NYTs t...,China tells Bush to stay out of other countrie...,Did World War III start today,Georgia Invades South Ossetia if Russia gets ...,AlQaeda Faces Islamist Backlash,Condoleezza Rice The US would not act to preve...,This is a busy day The European Union has app...,Georgia will withdraw 1000 soldiers from Iraq ...,Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis Georgia invades South Ossetia,Indian shoe manufactory And again in a serie...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexicos Kidnapping Surge,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1,2008-08-11,1,Why wont America and Nato help us If they wont...,Bush puts foot down on Georgian conflict,Jewish Georgian minister Thanks to Israeli tra...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks faked,What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in SOssetia blames ...,Welcome To World War IV Now In High Definition,Georgias move a mistake of monumental proporti...,Russia presses deeper into Georgia US says reg...,Abhinav Bindra wins first ever Individual Olym...,US ship heads for Arctic to define territory,Drivers in a Jerusalem taxi station threaten t...,The French Team is Stunned by Phelps and the 4...,Israel and the US behind the Georgian aggression,Do not believe TV neither Russian nor Georgian...,Riots are still going on in Montreal Canada be...,China to overtake US as largest manufacturer,War in South Ossetia PICS,Israeli Physicians Group Condemns State Torture,Russia has just beaten the United States over...,Perhaps the question about the Georgia Russia...,Russia is so much better at war,So this is what its come to trading sex for food,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2,2008-08-12,0,Remember that adorable 9yearold who sang at th...,Russia ends Georgia operation,If we had no sexual harassment we would have n...,AlQaeda is losing support in Iraq because of a...,Ceasefire in Georgia Putin Outmaneuvers the West,Why Microsoft and Intel tried to kill the XO 1...,Stratfor The RussoGeorgian War and the Balance...,Im Trying to Get a Sense of This Whole Georgia...,The US military was surprised by the timing an...,US Beats War Drum as Iran Dumps the Dollar,Gorbachev Georgian military attacked the South...,CNN use footage of Tskhinvali ruins to cover G...,Beginning a war as the Olympics were opening v...,55 pyramids as large as the Luxor stacked into...,The 11 Top Party Cities in the World,US troops still in Georgia did you know they w...,Why Russias response to Georgia was right,Gorbachev accuses US of making a serious blund...,Russia Georgia and NATO Cold War Two,Remember that adorable 62yearold who led your ...,War in Georgia The Israeli connection,All signs point to the US encouraging Georgia ...,Christopher King argues that the US and NATO a...,America The New Mexico,BBC NEWS AsiaPacific Extinction by man not c...,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
3,2008-08-13,0,US refuses Israel weapons to attac

In [143]:
#create new column with next row's open & close price as target and drop last row
df = full_df
df_open_tmp = pd.DataFrame(df.iloc[1:,-6])
df_close_tmp = pd.DataFrame(df.iloc[1:,-3])
new_row_1 = pd.DataFrame({"Open": [0]})
new_row_2 = pd.DataFrame({"Close": [0]})
new_row_open = df_open_tmp.append(new_row_1, ignore_index=True)
new_row_close = df_close_tmp.append(new_row_2, ignore_index=True)
df["p-Open"] = new_row_open
df["p-Close"] = new_row_close
df = df.iloc[:-1,:]

In [ ]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close,p-Open,p-Close
0,2008-08-08,0,Georgia downs two Russian warplanes as countri...,BREAKING Musharraf to be impeached,Russia Today Columns of troops roll into South...,Russian tanks are moving towards the capital o...,Afghan children raped with impunity UN officia...,150 Russian tanks have entered South Ossetia w...,Breaking Georgia invades South Ossetia Russia ...,The enemy combatent trials are nothing but a s...,Georgian troops retreat from S Osettain capita...,Did the US Prep Georgia for War with Russia,Rice Gives Green Light for Israel to Attack Ir...,AnnouncingClass Action Lawsuit on Behalf of Am...,SoRussia and Georgia are at war and the NYTs t...,China tells Bush to stay out of other countrie...,Did World War III start today,Georgia Invades South Ossetia if Russia gets ...,AlQaeda Faces Islamist Backlash,Condoleezza Rice The US would not act to preve...,This is a busy day The European Union has app...,Georgia will withdraw 1000 soldiers from Iraq ...,Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis Georgia invades South Ossetia,Indian shoe manufactory And again in a serie...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexicos Kidnapping Surge,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312,11729.669922,11782.349609
1,2008-08-11,1,Why wont America and Nato help us If they wont...,Bush puts foot down on Georgian conflict,Jewish Georgian minister Thanks to Israeli tra...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks faked,What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in SOssetia blames ...,Welcome To World War IV Now In High Definition,Georgias move a mistake of monumental proporti...,Russia presses deeper into Georgia US says reg...,Abhinav Bindra wins first ever Individual Olym...,US ship heads for Arctic to define territory,Drivers in a Jerusalem taxi station threaten t...,The French Team is Stunned by Phelps and the 4...,Israel and the US behind the Georgian aggression,Do not believe TV neither Russian nor Georgian...,Riots are still going on in Montreal Canada be...,China to overtake US as largest manufacturer,War in South Ossetia PICS,Israeli Physicians Group Condemns State Torture,Russia has just beaten the United States over...,Perhaps the question about the Georgia Russia...,Russia is so much better at war,So this is what its come to trading sex for food,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,11781.700195,11642.469727
2,2008-08-12,0,Remember that adorable 9yearold who sang at th...,Russia ends Georgia operation,If we had no sexual harassment we would have n...,AlQaeda is losing support in Iraq because of a...,Ceasefire in Georgia Putin Outmaneuvers the West,Why Microsoft and Intel tried to kill the XO 1...,Stratfor The RussoGeorgian War and the Balance...,Im Trying to Get a Sense of This Whole Georgia...,The US military was surprised by the timing an...,US Beats War Drum as Iran Dumps the Dollar,Gorbachev Georgian military attacked the South...,CNN use footage of Tskhinvali ruins to cover G...,Beginning a war as the Olympics were opening v...,55 pyramids as large as the Luxor stacked into...,The 11 Top Party Cities in the World,US troops still in Georgia did you know they w...,Why Russias response to Georgia was right,Gorbachev accuses US of making a serious blund...,Russia Georgia and NATO Cold War Two,Remember that adorable 62yearold who led your ...,War in Georgia The Israeli connection,All signs point to the US encouraging Georgia ...,Christopher King argues that the US and NATO a...,America The New Mexico,BBC NEWS AsiaPacific Extinction by man not c...,11781.700195,11782.349609,11601.519531,11642.469727,17359

In [144]:
#spliting sentiment,stock price,target stock price for model
df_num, df_y, df_sen = df.iloc[:,-8:-2],df.iloc[:,-2:], df.iloc[:,2:-8]
# if you want label to predict uncomment this line of code. In this notebook I will not consider current days label to predict next day's change
# df_y.insert(0,'Label',df.iloc[:,1])

print(df_num.head(),df_num.shape,'\n',df_sen,df_sen.shape,'\n',df_y,df_y.shape)


           Open          High  ...     Volume     Adj Close
0  11432.089844  11759.959961  ...  212830000  11734.320312
1  11729.669922  11867.110352  ...  183190000  11782.349609
2  11781.700195  11782.349609  ...  173590000  11642.469727
3  11632.809570  11633.780273  ...  182550000  11532.959961
4  11532.070312  11718.280273  ...  159790000  11615.929688

[5 rows x 6 columns] (1988, 6) 
                                                    Top1  ...                                              Top25
0     Georgia downs two Russian warplanes as countri...  ...               No Help for Mexicos Kidnapping Surge
1     Why wont America and Nato help us If they wont...  ...   So this is what its come to trading sex for food
2     Remember that adorable 9yearold who sang at th...  ...  BBC NEWS  AsiaPacific  Extinction by man not c...
3       US refuses Israel weapons to attack Iran report  ...  2006 Nobel laureate Aleksander Solzhenitsyn ac...
4     All the experts admit that we should leg

In [145]:
 df_y['p-change'] = df_y['p-Close']-df_y['p-Open']

In [146]:
pd.DataFrame(df_y)
del df_y['p-Close']
del df_y['p-Open']
df_y.head()

,p-change
0,52.679687
1,-139.230468
2,-99.849609
3,83.859376
4,48.690430


#Use sentence-transformers&DNN to convert news article to score(sentiment analysis)

#### 1. Use word embedding from pretrained BERT => flatten df_sen and embed every sentence => reshape df_sen for input
#### 2. Use RNN to output SEN_ANLYSIS_DIM from embedded df_sen
#### 3. Use RNN output as INPUT_B and also use df_num as INPUT_A for second RNN => output p-change

##### hyperparams : SEN_ANLYSIS_DIM

In [147]:
import tensorflow as tf
import tensorflow.keras.layers as L

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

In [149]:
df_sen_flatten = pd.DataFrame(df_sen).to_numpy().reshape(df_sen.shape[0]*df_sen.shape[1],1)

In [150]:
embedded_sentences=[]
for txt in df_sen_flatten:
  sentence_embeddings = model.encode(txt)
  embedded_sentences.append(sentence_embeddings)

In [151]:
embedded_sentences = np.array(embedded_sentences)
embedded_sentences = embedded_sentences.reshape((df_sen.shape[0],df_sen.shape[1],1024))

In [152]:
print(embedded_sentences.shape,embedded_sentences.dtype)

(1988, 25, 1024) float32


In [237]:
SEN_ANLYSIS_DIM = 64

input_A = L.Input(shape=(None,6), name='deep_input_num') #df_num
#batchnorm1 = L.BatchNormalization()(input_A)
hidden1 = L.Conv1D(SEN_ANLYSIS_DIM,kernel_size=1)(input_A)
input_B = L.Input(shape=(embedded_sentences.shape[1],embedded_sentences.shape[-1]), name="deep_input_sen",dtype='float32') #embedded_sentences
#batchnorm2 = L.BatchNormalization()(input_B)
hidden2 = L.Dropout(0.2)(input_B)
hidden3 = L.Dense(128,activation='tanh')(hidden2)
hidden4 = L.Dense(SEN_ANLYSIS_DIM)(hidden3)
concat = L.concatenate([hidden1,hidden4],axis=1)
hidden5 = L.LSTM(32,return_sequences=True)(concat)
hidden6 = L.LSTM(64)(hidden5)
output = L.Dense(1,activation='linear',name='output')(hidden6)
fin_model = tf.keras.Model(inputs=[input_A,input_B],outputs=[output])

In [154]:
#split data to train/dev
sen_train, sen_dev = train_test_split(embedded_sentences,test_size=0.1, shuffle=False)
num_train, num_dev = train_test_split(df_num,test_size=0.1, shuffle=False)
y_train,y_dev = train_test_split(df_y,test_size=0.1, shuffle=False)
num_train = tf.expand_dims(num_train,1)
num_dev = tf.expand_dims(num_dev,1)

In [192]:
print(num_train.shape,sen_train.shape)

(1789, 1, 6) (1789, 25, 1024)


In [156]:
df_sen.iloc[0,0]

'Georgia downs two Russian warplanes as countries move to brink of war'

In [210]:
sentence_embeddings_1 = model.encode('Georgia downs two Russian warplanes as countries move to brink of war')
sentence_embeddings_2 = embedded_sentences[0,0,:]
print(sentence_embeddings_1,'\n',sentence_embeddings_2)

[-0.06316123 -0.1335      0.99072087 ... -0.40782028 -0.5005878
 -1.6716199 ] 
 [-0.06316123 -0.1335      0.99072087 ... -0.40782028 -0.5005878
 -1.6716199 ]


In [240]:
tf.keras.backend.clear_session()

fin_model.compile(optimizer='Adam',loss='MSE',metrics=['accuracy','mse'])
history = fin_model.fit(x=[num_train,sen_train],y=y_train,batch_size=32,epochs=1000,verbose=2,validation_split=0.1)

Epoch 1/1000
51/51 - 1s - loss: 2720.5034 - accuracy: 6.2112e-04 - mse: 2720.5034 - val_loss: 31434.0566 - val_accuracy: 0.0000e+00 - val_mse: 31434.0566
Epoch 2/1000
51/51 - 0s - loss: 2794.3005 - accuracy: 6.2112e-04 - mse: 2794.3005 - val_loss: 32362.1738 - val_accuracy: 0.0000e+00 - val_mse: 32362.1738
Epoch 3/1000
51/51 - 0s - loss: 2764.1597 - accuracy: 0.0000e+00 - mse: 2764.1597 - val_loss: 31619.3457 - val_accuracy: 0.0000e+00 - val_mse: 31619.3457
Epoch 4/1000
51/51 - 0s - loss: 2707.9956 - accuracy: 0.0000e+00 - mse: 2707.9956 - val_loss: 30768.4980 - val_accuracy: 0.0000e+00 - val_mse: 30768.4980
Epoch 5/1000
51/51 - 0s - loss: 2705.6499 - accuracy: 0.0000e+00 - mse: 2705.6499 - val_loss: 31385.7148 - val_accuracy: 0.0000e+00 - val_mse: 31385.7148
Epoch 6/1000
51/51 - 0s - loss: 2583.5979 - accuracy: 0.0000e+00 - mse: 2583.5979 - val_loss: 32417.5059 - val_accuracy: 0.0000e+00 - val_mse: 32417.5059
Epoch 7/1000
51/51 - 0s - loss: 2697.8372 - accuracy: 6.2112e-04 - mse: 2697

In [241]:
plus_minus = []
for i in range(y_dev.shape[0]):
  if (y_dev.values[i]>0 and fin_model.predict(x=[tf.expand_dims(num_dev[i],0),tf.expand_dims(sen_dev[i],0)])>0) or (y_dev.values[i]<0 and fin_model.predict(x=[tf.expand_dims(num_dev[i],0),tf.expand_dims(sen_dev[i],0)])<0):
    plus_minus.append('Success')
  else:
    plus_minus.append('Fail')
print('Accuracy of predicting +/- is ',(plus_minus.count('Success')*100)/len(plus_minus),'%')

Accuracy of predicting +/- is  55.27638190954774 %


In [242]:
fin_model.predict(x=[tf.expand_dims(num_dev[0],0),tf.expand_dims(sen_dev[0],0)])

array([[-16.634369]], dtype=float32)

In [244]:
y_dev

,p-change
1789,-290.160156
1790,104.089844
1791,-146.979492
1792,-52.919922
1793,-55.790040
...,...
1983,-214.970704
1984,219.210937
1985,238.660157
1986,217.230468


In [243]:
y_train

,p-change
0,52.679687
1,-139.230468
2,-99.849609
3,83.859376
4,48.690430
...,...
1784,102.689453
1785,-79.899414
1786,217.269531
1787,140.439453
